# SCF Comparison Between Experimental, FEA, and Parameteric Equation  Results

In [ ]:
import pandas as pd
import matplotlib
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
matplotlib.style.use('ggplot')

In [ ]:
def linear_func(xdata, m):
    return m*xdata

In [ ]:
df = pd.read_csv('result_summary_combined.csv', na_values=['NaN'])

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df)

In [ ]:
def make_plot(x1, y1, x2, y2, popt_1, popt_2,ymax, x_axis, y_axis='FEA'):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    plt.scatter(x1.values,
                y1.values,
                color='r',
                label='FEA/1')
    plt.scatter(x2.values,
                y2.values,
                color='b',
                label='FEA/2')
    plt.plot([0,100],[0,100],color='k')
    plt.plot(x1.values, 
             linear_func(x1.values, *popt_1), 
             'r')
    plt.plot(x2.values, 
             linear_func(x2.values, *popt_2), 
             'b')
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    ax.set_aspect('equal')
    ax.set_xlim(0,ymax)
    ax.set_ylim(0,ymax)
    plt.legend()
    return fig

In [ ]:
def make_plot_experiment(x1, y1, x2, y2, popt_1, popt_2, ymax):
    fig = make_plot(x1, y1, x2, y2, popt_1, popt_2,ymax, 'Experiment')
    return fig

In [ ]:
def make_plot_experiment_efthymiou(x, y1, popt_1, ymax):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    plt.scatter(x.values,
                y1.values,
                color='r',
                label='Efthymiou')
    plt.plot([0,100],[0,100],color='k')
    plt.plot(x.values, 
             linear_func(x.values, *popt_1), 
             'r')
    plt.xlabel('Experiment')
    plt.ylabel('Efthymiou')
    ax.set_aspect('equal')
    ax.set_xlim(0,ymax)
    ax.set_ylim(0,ymax)
    plt.legend()
    return fig

In [ ]:
def make_plot_efthymiou(x, y1, y2, popt_1, popt_2, ymax):
    fig = make_plot(x, y1, y2, popt_1, popt_2,ymax, 'Efthymiou')
    return fig

In [ ]:
def outliers_iqr(ys):
    
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    #print(lower_bound)
    upper_bound = quartile_3 + (iqr * 1.5)
    #print(upper_bound)
    return np.argwhere((ys > upper_bound) | (ys < lower_bound))

In [ ]:
def process_case(lc):
    '''
    lc = 'AxialChordSaddle', etc.
    '''
    experiment = df.loc[df['Source'] == 'Experiment'][lc]
    efthy_exp = df.loc[df['Source'] == 'Efthymiou'][lc]
    fea = df.loc[df['Source'] == 'FEA'][lc]
    fea_2 = df.loc[df['Source'] == 'FEA/2'][lc]
    # Verify the lengths are equal
    assert len( set( [len(experiment), 
                      len(efthy_exp),
                      len(fea),
                      len(fea_2)]) ) == 1
    # Create dataframe from columns
    src = {'Experiment':experiment.values,
           'Efthymiou':efthy_exp.values,
           'FEA':fea.values,
           'FEA/2':fea_2.values}
    src_df = pd.DataFrame(data=src)
    #with pd.option_context('display.max_rows', None):
    #    print(src_df)
    
    # Process FEA
    
    # Drop rows with NaN
    fea_df = src_df.dropna(subset=['Experiment','FEA'])
    # Reindex
    fea_df = fea_df.reset_index(drop=True)
        
    # Detect outliers using inter-quartile range method
    indices_fea = outliers_iqr(fea_df['Experiment'].values - 
                               fea_df['FEA'].values)
    
    # Drop outliers
    fea_df = fea_df.drop(indices_fea.flatten())
    # Reindex
    fea_df = fea_df.reset_index(drop=True)

    # Process FEA/2
    
    # Drop rows with NaN
    fea_2_df = src_df.dropna(subset=['Experiment','FEA/2'])
    # Reindex
    fea_2_df = fea_2_df.reset_index(drop=True)
        
    # Detect outliers using inter-quartile range method
    indices_fea_2 = outliers_iqr(fea_2_df['Experiment'].values - 
                                 fea_2_df['FEA/2'].values)
    
    # Drop outliers
    fea_2_df = fea_2_df.drop(indices_fea_2.flatten())
    # Reindex
    fea_2_df = fea_2_df.reset_index(drop=True)

    # Process Efthymiou
    
    # Drop rows with NaN
    efthy_df = src_df.dropna(subset=['Experiment','Efthymiou'])
    # Reindex
    efthy_df = efthy_df.reset_index(drop=True)
        
    # Detect outliers using inter-quartile range method
    indices_efthy = outliers_iqr(efthy_df['Experiment'].values - 
                                 efthy_df['Efthymiou'].values)
    
    # Drop outliers
    efthy_df = efthy_df.drop(indices_efthy.flatten())
    # Reindex
    efthy_df = efthy_df.reset_index(drop=True)
    
    d = {'fea':fea_df,
         'fea_2':fea_2_df,
         'efthy':efthy_df}
    
    return d

In [ ]:
def fit(d):
    popt_1, pcov_1 = curve_fit(linear_func, 
                               d['fea']['Experiment'].values, 
                               d['fea']['FEA'].values)
    popt_2, pcov_2 = curve_fit(linear_func, 
                               d['fea_2']['Experiment'].values, 
                               d['fea_2']['FEA/2'].values)
    popt_3, pcov_3 = curve_fit(linear_func, 
                               d['efthy']['Experiment'].values, 
                               d['efthy']['Efthymiou'].values)
    return ((popt_1, popt_2, popt_3),
            (pcov_1, pcov_2, pcov_3))

## Axial

### Chord

#### Saddle

In [ ]:
d_axial_chord_saddle = process_case('AxialChordSaddle')

In [ ]:
popt_axial_chord_saddle, pcov_axial_chord_saddle = fit(d_axial_chord_saddle)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_axial_chord_saddle[0][0]))
perr = np.sqrt(np.diag(pcov_axial_chord_saddle[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_axial_chord_saddle[1][0]))
perr = np.sqrt(np.diag(pcov_axial_chord_saddle[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_axial_chord_saddle[2][0]))
perr = np.sqrt(np.diag(pcov_axial_chord_saddle[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_axial_chord_saddle['fea']['Experiment'],
                           d_axial_chord_saddle['fea']['FEA'],
                           d_axial_chord_saddle['fea_2']['Experiment'],
                           d_axial_chord_saddle['fea_2']['FEA/2'], 
                           popt_axial_chord_saddle[0], 
                           popt_axial_chord_saddle[1],
                           20)
fig.savefig('AxialChordSaddle_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Chord/Saddle')

In [ ]:
fig = make_plot_experiment_efthymiou(d_axial_chord_saddle['efthy']['Experiment'],
                                     d_axial_chord_saddle['efthy']['Efthymiou'], 
                                     popt_axial_chord_saddle[2],
                                     20)
fig.savefig('AxialChordSaddle_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Chord/Saddle')

#### Crown

In [ ]:
d_axial_chord_crown = process_case('AxialChordCrown')

In [ ]:
popt_axial_chord_crown, pcov_axial_chord_crown = fit(d_axial_chord_crown)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_axial_chord_crown[0][0]))
perr = np.sqrt(np.diag(pcov_axial_chord_crown[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_axial_chord_crown[1][0]))
perr = np.sqrt(np.diag(pcov_axial_chord_crown[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_axial_chord_crown[2][0]))
perr = np.sqrt(np.diag(pcov_axial_chord_crown[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_axial_chord_crown['fea']['Experiment'],
                           d_axial_chord_crown['fea']['FEA'],
                           d_axial_chord_crown['fea_2']['Experiment'],
                           d_axial_chord_crown['fea_2']['FEA/2'], 
                           popt_axial_chord_crown[0], 
                           popt_axial_chord_crown[1],
                           7)
fig.savefig('AxialChordCrown_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Chord/Crown')

In [ ]:
fig = make_plot_experiment_efthymiou(d_axial_chord_crown['efthy']['Experiment'],
                                     d_axial_chord_crown['efthy']['Efthymiou'], 
                                     popt_axial_chord_crown[2],
                                     8)
fig.savefig('AxialChordCrown_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Chord/Crown')

### Brace

#### Saddle

In [ ]:
d_axial_brace_saddle = process_case('AxialBraceSaddle')

In [ ]:
popt_axial_brace_saddle, pcov_axial_brace_saddle = fit(d_axial_brace_saddle)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_axial_brace_saddle[0][0]))
perr = np.sqrt(np.diag(pcov_axial_brace_saddle[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_axial_brace_saddle[1][0]))
perr = np.sqrt(np.diag(pcov_axial_brace_saddle[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_axial_brace_saddle[2][0]))
perr = np.sqrt(np.diag(pcov_axial_brace_saddle[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_axial_brace_saddle['fea']['Experiment'],
                           d_axial_brace_saddle['fea']['FEA'],
                           d_axial_brace_saddle['fea_2']['Experiment'],
                           d_axial_brace_saddle['fea_2']['FEA/2'], 
                           popt_axial_brace_saddle[0], 
                           popt_axial_brace_saddle[1],
                           16)
fig.savefig('AxialBraceSaddle_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Brace/Saddle')

In [ ]:
fig = make_plot_experiment_efthymiou(d_axial_brace_saddle['efthy']['Experiment'],
                                     d_axial_brace_saddle['efthy']['Efthymiou'], 
                                     popt_axial_brace_saddle[2],
                                     14)
fig.savefig('AxialBraceSaddle_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Brace/Saddle')

#### Crown

In [ ]:
d_axial_brace_crown = process_case('AxialBraceCrown')

In [ ]:
popt_axial_brace_crown, pcov_axial_brace_crown = fit(d_axial_brace_crown)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_axial_brace_crown[0][0]))
perr = np.sqrt(np.diag(pcov_axial_brace_crown[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_axial_brace_crown[1][0]))
perr = np.sqrt(np.diag(pcov_axial_brace_crown[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_axial_brace_crown[2][0]))
perr = np.sqrt(np.diag(pcov_axial_brace_crown[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_axial_brace_crown['fea']['Experiment'],
                           d_axial_brace_crown['fea']['FEA'],
                           d_axial_brace_crown['fea_2']['Experiment'],
                           d_axial_brace_crown['fea_2']['FEA/2'], 
                           popt_axial_brace_crown[0], 
                           popt_axial_brace_crown[1],
                           4)
fig.savefig('AxialBraceCrown_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Brace/Crown')

In [ ]:
fig = make_plot_experiment_efthymiou(d_axial_brace_crown['efthy']['Experiment'],
                                     d_axial_brace_crown['efthy']['Efthymiou'], 
                                     popt_axial_brace_crown[2],
                                     4)
fig.savefig('AxialBraceCrown_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('Axial/Brace/Crown')

## OPB

### Chord

#### Saddle

In [ ]:
d_opb_chord_saddle = process_case('OPBChordSaddle')

In [ ]:
popt_opb_chord_saddle, pcov_opb_chord_saddle = fit(d_opb_chord_saddle)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_opb_chord_saddle[0][0]))
perr = np.sqrt(np.diag(pcov_opb_chord_saddle[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_opb_chord_saddle[1][0]))
perr = np.sqrt(np.diag(pcov_opb_chord_saddle[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_opb_chord_saddle[2][0]))
perr = np.sqrt(np.diag(pcov_opb_chord_saddle[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_opb_chord_saddle['fea']['Experiment'],
                           d_opb_chord_saddle['fea']['FEA'],
                           d_opb_chord_saddle['fea_2']['Experiment'],
                           d_opb_chord_saddle['fea_2']['FEA/2'], 
                           popt_opb_chord_saddle[0], 
                           popt_opb_chord_saddle[1],
                           20)
fig.savefig('OPBChordSaddle_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('OPB/Chord/Saddle')

In [ ]:
fig = make_plot_experiment_efthymiou(d_opb_chord_saddle['efthy']['Experiment'],
                                     d_opb_chord_saddle['efthy']['Efthymiou'], 
                                     popt_opb_chord_saddle[2],
                                     25)
fig.savefig('OPBChordSaddle_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('OPB/Chord/Saddle')

### Brace

#### Saddle

In [ ]:
d_opb_brace_saddle = process_case('OPBBraceSaddle')

In [ ]:
popt_opb_brace_saddle, pcov_opb_brace_saddle = fit(d_opb_brace_saddle)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_opb_brace_saddle[0][0]))
perr = np.sqrt(np.diag(pcov_opb_brace_saddle[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_opb_brace_saddle[1][0]))
perr = np.sqrt(np.diag(pcov_opb_brace_saddle[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_opb_brace_saddle[2][0]))
perr = np.sqrt(np.diag(pcov_opb_brace_saddle[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_opb_brace_saddle['fea']['Experiment'],
                           d_opb_brace_saddle['fea']['FEA'],
                           d_opb_brace_saddle['fea_2']['Experiment'],
                           d_opb_brace_saddle['fea_2']['FEA/2'], 
                           popt_opb_brace_saddle[0], 
                           popt_opb_brace_saddle[1],
                           15)
fig.savefig('OPBBraceSaddle_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('OPB/Brace/Saddle')

In [ ]:
fig = make_plot_experiment_efthymiou(d_opb_brace_saddle['efthy']['Experiment'],
                                     d_opb_brace_saddle['efthy']['Efthymiou'], 
                                     popt_opb_brace_saddle[2],
                                     12)
fig.savefig('OPBBraceSaddle_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('OPB/Brace/Saddle')

## IPB

### Chord

#### Crown

In [ ]:
d_ipb_chord_crown = process_case('IPBChordCrown')

In [ ]:
popt_ipb_chord_crown, pcov_ipb_chord_crown = fit(d_ipb_chord_crown)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_ipb_chord_crown[0][0]))
perr = np.sqrt(np.diag(pcov_ipb_chord_crown[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_ipb_chord_crown[1][0]))
perr = np.sqrt(np.diag(pcov_ipb_chord_crown[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_ipb_chord_crown[2][0]))
perr = np.sqrt(np.diag(pcov_ipb_chord_crown[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_ipb_chord_crown['fea']['Experiment'],
                           d_ipb_chord_crown['fea']['FEA'],
                           d_ipb_chord_crown['fea_2']['Experiment'],
                           d_ipb_chord_crown['fea_2']['FEA/2'], 
                           popt_ipb_chord_crown[0], 
                           popt_ipb_chord_crown[1],
                           7)
fig.savefig('IPBChordCrown_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('IPB/Chord/Crown')

In [ ]:
fig = make_plot_experiment_efthymiou(d_ipb_chord_crown['efthy']['Experiment'],
                                     d_ipb_chord_crown['efthy']['Efthymiou'], 
                                     popt_ipb_chord_crown[2],
                                     7)
fig.savefig('IPBChordCrown_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('IPB/Chord/Crown')

### Brace

#### Crown

In [ ]:
d_ipb_brace_crown = process_case('IPBBraceCrown')

In [ ]:
popt_ipb_brace_crown, pcov_ipb_brace_crown = fit(d_ipb_brace_crown)

In [ ]:
print('Slope FEA/1 vs experiment: {:0.2f}'.format(popt_ipb_brace_crown[0][0]))
perr = np.sqrt(np.diag(pcov_ipb_brace_crown[0]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope FEA/2 vs experiment: {:0.2f}'.format(popt_ipb_brace_crown[1][0]))
perr = np.sqrt(np.diag(pcov_ipb_brace_crown[1]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
print('Slope: Efthymiou vs experiment: {:0.2f}'.format(popt_ipb_brace_crown[2][0]))
perr = np.sqrt(np.diag(pcov_ipb_brace_crown[2]))[0]
print('One standard deviation error on the slope: {:0.2f}'.format(perr))

In [ ]:
fig = make_plot_experiment(d_ipb_brace_crown['fea']['Experiment'],
                           d_ipb_brace_crown['fea']['FEA'],
                           d_ipb_brace_crown['fea_2']['Experiment'],
                           d_ipb_brace_crown['fea_2']['FEA/2'], 
                           popt_ipb_brace_crown[0], 
                           popt_ipb_brace_crown[1],
                           4)
fig.savefig('IPBBraceCrown_fea_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('FEA vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('IPB/Brace/Crown')

In [ ]:
fig = make_plot_experiment_efthymiou(d_ipb_brace_crown['efthy']['Experiment'],
                                     d_ipb_brace_crown['efthy']['Efthymiou'], 
                                     popt_ipb_brace_crown[2],
                                     5)
fig.savefig('IPBBraceCrown_efthy_vs_exp.pdf', bbox_inches='tight')
fig.suptitle('Efthymiou vs. Experimental SCF', fontsize=14, fontweight='bold')
fig.axes[0].set_title('IPB/Brace/Crown')